In [1]:
import mlflow
import torch
from torch.utils.data import DataLoader
from glob import glob
import os
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from components.mymodel import load_model, get_model
from components.helper import train

In [2]:
#We can check whether we have gpu
DEVICE = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
# DEVICE = 'cpu'
print("Device: ", DEVICE)

Device:  cuda:0


In [3]:
import torch
from torch.utils.data import Dataset
from glob import glob
import os
import pandas as pd
from torchvision import io
from enum import Enum

class Devices(Enum):
    iphone = 'Iphone'
    oppo = 'Oppo'
    redmi = 'Redmi'
    samsung = 'Samsung'
    all = '*'

class Environments(Enum):
    indoor = 'Indoor'
    outdoor = 'Outdoor'
    all = '*'

class Imageset(Enum):
    om = 'OM'
    p = 'P'

class SoilDataset_bigset(Dataset):
    def __init__(self, imageset:Imageset, device:Devices, environment:Environments, transform=None):
        # BasePath of the dataset
        dataset_path:str = './dataset/bigset/'
        assert os.path.exists(dataset_path), f"{dataset_path=} is not exist."
        # Inside this path there must be a list of folders arange by mobile phone. Use device enum.
        # Inside those mobile phone are 2 folders indicate the environment the image was taken in. Use environment enum.
        image_folder = os.path.join(dataset_path, imageset.value, device.value, environment.value)
        self.imgs = glob(os.path.join(image_folder,'*/*'))
        print(f"Found {len(self.imgs)} images in {image_folder}.")

        # Load csv file for lookup the target value
        target_path:str = os.path.join(dataset_path,imageset.value,'meta.csv')
        self.target_df = pd.read_csv(target_path, index_col='id')

        self.signature = os.path.join(imageset.value,device.value,environment.value)
        self.transform = transform

    def get_target(self, img_path:str) -> float:
        assert len(img_path.split('/')) == 8, f"Expect img_path to have 8 folders but got {img_path=}"
        target_id = int(img_path.split('/')[6])
        return self.target_df.loc[target_id] # type:ignore
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        y = self.get_target(img_path=img_path)
        y = torch.tensor(y)
        X = io.read_image(img_path)
        if self.transform:
            X = self.transform(X)
        return X.float(), y.float(), img_path

In [4]:
def train_model(model, dataset:SoilDataset_bigset, epochs:int, lr:float, save_path:str):
    loader = DataLoader(dataset=dataset, batch_size=200, shuffle=True, num_workers=30)
    model, train_losses = train(model, loader, epochs, lr, DEVICE, save_path)
    plt.plot(train_losses)
    plt.title(dataset.signature)
    plt.show()
    return model, train_losses

In [5]:
mlflow.set_tracking_uri("https://web-mlflow.akraradets.duckdns.org")
mlflow.get_tracking_uri()
mlflow.set_experiment(experiment_name='Soil')
mlflow.start_run(run_name="OM Bigset-resizefirst")

<ActiveRun: >

In [6]:
# from PIL import Image

# img = Image.open(dataset.imgs[100])
# # dataset.imgs[100]
# img
# plt.imshow( img_array )

In [7]:
image_set = Imageset.om
device = Devices.all
environment = Environments.all

train_losses = []
# mobilenet_v3_large
# resnet50
# efficientnet_v2_l
# alexnet

model_name = mlflow.log_param("model_name", 'alexnet')
weight_path = f'./weight/{model_name}'

params:dict = dict({
    'epochs': 1000,
    'lr': 0.001,
    'Imageset': image_set.value,
    'Device': device.value,
    'Environment': environment.value
})

mlflow.log_params(params)

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(350),
    transforms.CenterCrop(224),
    # transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = SoilDataset_bigset(imageset=Imageset.om, device=Devices.all, environment=Environments.all, transform=preprocess)

weight_path = os.path.join(weight_path,dataset.signature)
if(os.path.exists(weight_path) == False):
    os.makedirs(weight_path)

save_path =  os.path.join(weight_path,f"{model_name}.pth")
# Continue
# model = get_model(model_name=model_name, image_set=image_set)
# From scratch
model = load_model(model_name=model_name)
# model = torch.compile(model)
model, train_loss = train_model(model, dataset=dataset, save_path=save_path, epochs=params['epochs'], lr=params['lr'])
train_losses.append(train_loss)

Found 4047 images in ./dataset/bigset/OM/*/*.
44.703086376190186 0 tensor(730.9408)


2023/08/30 04:51:34 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/08/30 04:51:37 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


44.64417862892151 1 tensor(131.3284)


2023/08/30 04:52:25 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/08/30 04:52:27 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


45.230419397354126 2 tensor(87.2748)


...

2023/08/30 07:43:59 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/08/30 07:44:01 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


44.701757192611694 211 tensor(0.9117)


2023/08/30 07:44:48 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/08/30 07:44:50 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


44.88970232009888 212 tensor(1.0543)


2023/08/30 07:45:38 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/08/30 07:45:40 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.2+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
